In [1]:
import mne
from mne.datasets import sample
import numpy as np
from hdf5storage import loadmat, savemat 

In [2]:
# the raw file containing the channel location + types
raw_fname = './egi_raw.fif'
# The paths to Freesurfer reconstructions
subjects_dir ='../../archieve/subjects/'
subject = 'fsaverage' # change it to use freesurfer's bem

In [3]:
trans = './egi_trans.fif'

In [4]:
# Compute Source Space (surface)
src = mne.setup_source_space(subject, spacing='all', add_dist='patch',
                             subjects_dir=subjects_dir)

Setting up the source space with the following parameters:

SUBJECTS_DIR = c:\Users\zhouz\GitHub\MEG_EEG_Source_Localization\EEG_pipeline\..\..\archieve\subjects
Subject      = fsaverage
Surface      = white
Include all vertices

>>> 1. Creating the source space...

Loading c:\Users\zhouz\GitHub\MEG_EEG_Source_Localization\EEG_pipeline\..\..\archieve\subjects\fsaverage\surf\lh.white...
    Triangle neighbors and vertex normals...
loaded lh.white 163842/163842 selected to source space (all)

Loading c:\Users\zhouz\GitHub\MEG_EEG_Source_Localization\EEG_pipeline\..\..\archieve\subjects\fsaverage\surf\rh.white...
    Triangle neighbors and vertex normals...
loaded rh.white 163842/163842 selected to source space (all)

Calculating patch information (limit=0.0 mm)...
    Computing patch statistics...
    Patch information added...
    Computing patch statistics...
    Patch information added...
You are now one step closer to computing the gain matrix


In [5]:
conductivity = (0.3, 0.006, 0.3)  #  three layers for EEG
model = mne.make_bem_model(subject='fsaverage', ico=3,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 3th subdivision of an icosahedron (n_tri: 20480 -> 1280)
Going from 5th to 3th subdivision of an icosahedron (n_tri: 20480 -> 1280)
Going from 5th to 3th subdivision of an icosahedron (n_tri: 20480 -> 1280)
outer skin  CM is  -0.21 -19.38  -0.22 mm
outer skull CM is  -0.19 -19.35  -0.49 mm
inner skull CM is  -0.53 -21.10   6.21 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    1.6 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    5.5 mm
Surfaces passed the basic topology checks.
Complete.

Three-layer model surfaces loaded.
Computing the linear collocation solution...
    Matrix coefficients...
     

In [6]:
fwd = mne.make_forward_solution(raw_fname, trans=trans, src=src, bem=bem,
                                meg=False, eeg=True, mindist=5.0, n_jobs=2,
                                verbose=True)
print(fwd)
# It takes 1m27s

Source space          : <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=163842>, <surface (rh), n_vertices=163842, n_used=163842>] MRI (surface RAS) coords, subject 'fsaverage', ~71.5 MB>
MRI -> head transform : egi_trans.fif
Measurement data      : egi_raw.fif
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 327684 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999060 -0.029951 -0.031346      -0.53 mm
     0.027960  0.997677 -0.062124      21.78 mm
     0.033134  0.061189  0.997576      23.40 mm
     0.000000  0.000000  0.000000       1.00

Read 256 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Employing the head->MRI coordinate transform with the BEM model.
BEM model instance of ConductorModel is now set up

Source spaces are in head coordinates.
Checking th

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


    Found      0/123074 points outside using solid angles
    Total 163842/163842 points inside the surface
Interior check completed in 31612.1 ms
Checking surface interior status for 163842 points...
    Found  37548/163842 points inside  an interior sphere of radius   48.4 mm


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   31.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   31.4s finished


    Found      0/163842 points outside an exterior sphere of radius   97.8 mm
    Found      0/126294 points outside using surface Qhull


...

    Found      0/126294 points outside using solid angles
    Total 163842/163842 points inside the surface
Interior check completed in 25367.9 ms

Setting up for EEG...


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   25.2s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   25.2s finished


Computing EEG at 327684 source locations (free orientations)...


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   30.9s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   30.9s finished



Finished.
<Forward | MEG channels: 0 | EEG channels: 256 | Source space: Surface with 327684 vertices | Source orientation: Free>


In [7]:
print(f'Before: {src}')
print(f'After:  {fwd["src"]}')

Before: <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=163842>, <surface (rh), n_vertices=163842, n_used=163842>] MRI (surface RAS) coords, subject 'fsaverage', ~71.5 MB>
After:  <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=163842>, <surface (rh), n_vertices=163842, n_used=163842>] head coords, subject 'fsaverage', ~71.5 MB>


In [8]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Leadfield size : 256 sensors x 327684 dipoles


In [9]:
np.shape(leadfield)

(256, 327684)

In [10]:
outdict=dict()
outdict['leadfield']=leadfield
outdict['source_nn']=fwd_fixed['source_nn']
outdict['source_rr']=fwd_fixed['source_rr']
savemat('leadfield_nn_rr',outdict)

In [11]:
mne.write_forward_solution('surface_egi_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=True, verbose=None)

Overwriting existing file.
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


C:\Users\zhouz\AppData\Local\Temp\ipykernel_10480\2403303488.py:1: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  mne.write_forward_solution('surface_egi_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=True, verbose=None)
